# Подготовка данных

## Настройка и загрузка датасета

In [1]:
from pathlib import Path
from vk_mod.data import load_dataset


SAMPLE_LIMIT:int = 40000
preprocessed_path = Path('../data/preprocessed/dataset.csv')
images_dir = Path("..data/images")

df = load_dataset(preprocessed_path, sample_limit=SAMPLE_LIMIT)

c:\Users\ShapeCat\source\repos\ToxicPostDetection\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\ShapeCat\source\repos\ToxicPostDetection\src\vk_mod\data\loader.py:29: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path).fillna("")


## Небольшая статистика

In [2]:
has_text = df['text'].apply(lambda x: len(str(x).strip()) > 0)
has_image = df['image_path'].apply(lambda x: len(str(x).strip()) > 0)
print(f"""
      Статистика комбинаций данных:
      Только Текст: {len(df[has_text & ~has_image])}
      Только Картинка: {len(df[~has_text & has_image])}
      Только Текст+Картинка: {len(df[has_text & has_image])}
      Весь набор: {len(df)}
          """)
display(df.sample(10))


      Статистика комбинаций данных:
      Только Текст: 39422
      Только Картинка: 547
      Только Текст+Картинка: 31
      Весь набор: 40000
          


,text,toxic,image_path
226082,помогите пожалуйста найти собачка семьи! зима ...,0,
86696,действительно классссное!!!!!!!!!!!!,0,
142206,сука вонючая сдохни тварь,1,
190712,а окуня с «медалями» в ирмингера не ловил? а к...,0,
130761,а зачем ?!!,0,
105931,буду рад знакомству с тобой красавица. можем п...,0,
274609,самая прекрасная новость!,0,
39473,новый блокбастер люди в белом,0,
265085,пидор старый иди подмойся а то от тебя суки од...,1,
64886,толька суки продажные могут призывать народ на...,1,


## Формирование выборок

In [3]:
from sklearn.model_selection import train_test_split
from vk_mod.const import SEED


train_df, val_df = train_test_split(df, test_size=0.2, random_state=SEED)
train_df, test_df = train_test_split(train_df, test_size=0.1, random_state=SEED, shuffle=False)

# Обучение модели

In [4]:
from vk_mod.models import toxic_classificator

model_path = Path("../pretrained_models/multi.keras")

model = toxic_classificator.train_model(
    train_df,
    val_df,
    images_dir,
    epochs=100,
    save_path=model_path
)

Epoch 1/100
    900/Unknown 683s 736ms/step - accuracy: 0.8032 - loss: 0.4892

c:\Users\ShapeCat\source\repos\ToxicPostDetection\.venv\lib\site-packages\keras\src\trainers\epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


900/900 ━━━━━━━━━━━━━━━━━━━━ 846s 918ms/step - accuracy: 0.8032 - loss: 0.4891 - val_accuracy: 0.8581 - val_loss: 0.3282 - learning_rate: 1.0000e-04
Epoch 2/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 770s 855ms/step - accuracy: 0.8802 - loss: 0.2864 - val_accuracy: 0.9115 - val_loss: 0.2473 - learning_rate: 1.0000e-04
Epoch 3/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 763s 847ms/step - accuracy: 0.9362 - loss: 0.1716 - val_accuracy: 0.9224 - val_loss: 0.2311 - learning_rate: 1.0000e-04
Epoch 4/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 752s 836ms/step - accuracy: 0.9592 - loss: 0.1158 - val_accuracy: 0.9234 - val_loss: 0.2391 - learning_rate: 1.0000e-04
Epoch 5/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 754s 838ms/step - accuracy: 0.9715 - loss: 0.0841 - val_accuracy: 0.9236 - val_loss: 0.2504 - learning_rate: 5.0000e-05
Epoch 6/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 827s 919ms/step - accuracy: 0.9774 - loss: 0.0716 - val_accuracy: 0.9226 - val_loss: 0.2559 - learning_rate: 2.5000e-05
Epoch 7/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 809s 

# Оценка модели

## Оценка качества на разных комбинациях

In [5]:
toxic_classificator.evaluate_data_combinations(model, test_df, images_dir)

Данные: Только Текст
99/99 ━━━━━━━━━━━━━━━━━━━━ 64s 642ms/step - accuracy: 0.9219 - loss: 0.2191
Данные: Только Картинка
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 262ms/step - accuracy: 0.7703 - loss: 0.5841
Данные: Только Текст+Картинка
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step - accuracy: 0.7500 - loss: 0.2742
Данные: Весь набор
100/100 ━━━━━━━━━━━━━━━━━━━━ 62s 619ms/step - accuracy: 0.9196 - loss: 0.2238


## Проверка сохранения

In [7]:
import keras


predict_after_training = toxic_classificator.predict_from_file(model, "это что за п*здец!?!!!!!", "e621v32986.jpg")
print(f"After training: {predict_after_training} -> {predict_after_training > 0.5}")

model = keras.models.load_model(model_path)
predict_after_reloading =  toxic_classificator.predict_from_file(model, "это что за п*здец!?!!!!!", "e621v32986.jpg")
print(f"After reloading: {predict_after_reloading} -> {predict_after_reloading > 0.5}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
After training: 0.10666180402040482 -> False
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
After reloading: 0.10666180402040482 -> False


## Проверка картинки по url

In [14]:
toxic_classificator.predict_from_url(model, "", "https://sun9-35.userapi.com/impg/doJp2md6Q18pWWtXHNgeMxgbcWQ2qSwHKG-BPA/VkvhrKl89Vc.jpg?size=1280x1280&quality=95&sign=25cb6e551f5b588be0d9c4fb52e2e767&type=album")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step


0.6005491

## Контент смешанного характера

### Положительный текст и отрицательная картинка

In [15]:
toxic_classificator.predict_from_url(model, "положительный текст", "https://sun9-35.userapi.com/impg/doJp2md6Q18pWWtXHNgeMxgbcWQ2qSwHKG-BPA/VkvhrKl89Vc.jpg?size=1280x1280&quality=95&sign=25cb6e551f5b588be0d9c4fb52e2e767&type=album")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step


0.029985666

### Oтрицательная текст и положительная картинка

In [17]:
toxic_classificator.predict_from_url(model, "Что это за говно?!?!?!", "https://sun9-75.userapi.com/s/v1/if2/lP4YTYw5D4Ca1FYpQbgcZXhYQ-uI4Dv-Hg4TiYQ2l_9SDcqlpLqrqQQ5gCgjwmN0_sjj9YajX1BtDCdjP_hMVWmj.jpg?quality=96&as=32x24,48x36,72x54,108x81,160x120,240x180,360x270,480x360,540x405,640x480,720x540,1080x810,1280x960,1440x1080,2560x1920&from=bu&u=QCTk0zC8dlAqQg4g_uPSEn_dSMGvaqep7wUd5rG33y4&cs=807x605")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step


0.82072085